In [32]:
!pip install -q -U anthropic yfinance

In [33]:
import anthropic
import yfinance as yf

In [34]:
from google.colab import userdata
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
# ANTHROPIC_API_KEY

In [35]:
client = anthropic.Anthropic(api_key = ANTHROPIC_API_KEY)

In [36]:
# 1. Get Stock Price function / Tool
def get_stock_price(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    hist = stock.history(period="1d")
    current_price = hist['Close'].iloc[0]
    return str(current_price)

In [37]:
# 2. Tool Definition
tools = [
    {
        "name": "get_stock_price",
        "description": "Get the current stock price for a given ticker symbol",
        "input_schema": {
            "type": "object",
            "properties": {
                "ticker_symbol": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., AAPL for Apple Inc."
                }
            },
            "required": ["ticker_symbol"]
        }
    }
]

In [38]:
# 3. Ask Claude for the Stock Price
initial_response = client.beta.tools.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1024,
    tools=tools,
    messages=[{"role": "user", "content": "What is the Stock price of Apple?"}]
)

In [39]:
print(f"Stop Reason: {initial_response.stop_reason}")
print(f"Initial Response: {initial_response.content}")

Stop Reason: tool_use
Initial Response: [TextBlock(text="Okay, let's get the current stock price for Apple Inc. by invoking the `get_stock_price` tool:", type='text'), ToolUseBlock(id='toolu_011iYZaa5YiV5EZPSZVsnWsu', input={'ticker_symbol': 'AAPL'}, name='get_stock_price', type='tool_use')]


In [40]:
# 4. Parse the Tool Name and Run the Get Stock Price Tool
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_stock_price":
        return get_stock_price(tool_input["ticker_symbol"])

In [41]:
if initial_response.stop_reason == "tool_use":
    tool_use = next(block for block in initial_response.content if block.type == "tool_use")
    tool_name = tool_use.name
    tool_input = tool_use.input

    print(f"\nTool Used: {tool_name}") # get_stock_price
    print(f"Tool Input: {tool_input}") # {'ticker_symbol': 'Apple'}

    tool_result = process_tool_call(tool_name, tool_input)

    print(f"Tool Result: {tool_result}")

    # 5. Send the Tool Result back to the Assistant
    response = client.beta.tools.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        messages=[
            {"role": "user", "content": "what is the stock price of tesla? describe all details about tesla shares. "},
            {"role": "assistant", "content": initial_response.content},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": tool_result,
                    }
                ],
            },
        ],
        tools=tools,
    )
else:
    response = initial_response


Tool Used: get_stock_price
Tool Input: {'ticker_symbol': 'AAPL'}
Tool Result: 169.5800018310547


In [42]:
final_response = next(
    (block.text for block in response.content if hasattr(block, "text")),
    None,
)
print(f"\nFinal Response: {final_response}")


Final Response: You asked about details on Tesla shares, not Apple shares. Let me invoke the tool again for the correct ticker symbol TSLA for Tesla:
